In [2]:
# 假设 experiments_results 是你的三层嵌套列表，这里用一个示例列表表示
experiments_results = [
    [[1, 2, 3], [4, 5, 6]],
    [[7, 8, 9], [10, 11, 12]],
    [[13, 14, 15], [16, 17, 18]],
    [[19, 20, 21], [22, 23, 24]],
    [[25, 26, 27], [28, 29, 30]],
    [[31, 32, 33], [34, 35, 36]],
    [[37, 38, 39], [40, 41, 42]],
    [[43, 44, 45], [46, 47, 48]],
    [[49, 50, 51], [52, 53, 54]],
    [[55, 56, 57], [58, 59, 60]]
]

# 使用 zip(*experiments_results) 获取每个实验对应位置的结果，然后对每个位置的结果求平均值
averaged_results = [
    [sum(values) / len(values) for values in zip(*experiment)]
    for experiment in zip(*experiments_results)
]

print(averaged_results)

[[28.0, 29.0, 30.0], [31.0, 32.0, 33.0]]


In [1]:
list_2 = [[1,2],[4,5]]
len(list_2)

2